In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pyterrier as pt
if not pt.started():
    pt.init()

import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', False)

from tqdm import tqdm
tqdm.pandas()

# import pyt_splade
# splade = pyt_splade.Splade()



In [ ]:
dataset_name = 'msmarco-passage'
dataset = pt.get_dataset(f'irds:{dataset_name}')

# dataset2 = ir_datasets.load("msmarco-passage/dev")
# df2 = pd.DataFrame(pt.tqdm(dataset2.docs_iter()))

df = pd.DataFrame(dataset.get_corpus_iter(verbose=True))

In [ ]:
df.shape

In [ ]:
df2.shape

In [ ]:
import ir_datasets
dataset = ir_datasets.load("msmarco-passage/dev")
count = 0
for doc in dataset.docs_iter():
    print(doc)
    count +=1
    if count >5:
        break

In [ ]:
# ms_docids = {key['docno']:0 for key in dataset.get_corpus_iter(verbose=True)}

In [ ]:
import json

def save(dict, file):
    with open(file,'w') as f:
        json.dump(dict, f)


In [ ]:
def load(file):
    with open(file,'r') as f:
        ms_docids = json.load(f)
    return ms_docids

In [ ]:
# savedisk(ms_docids, './ms_docids.json')

In [ ]:
# for k,v in ms_docids.items():
#     print(type(k),type(v))

In [ ]:
#################### bm25 ###########################

In [ ]:

def calc_rtr_score(df, docid_dict, savename):
    for qid in topics['qid']:
        D = df[df['qid'] == np.int64(qid)]
        for dno in D['docno']:
            rank =  D[D['docno'] == dno]['rank'].values[0]
            score = 100/np.log(rank + 2) # plus 2 because the ranks start from zero
            docid_dict[str(dno)] += score
    
    save(docid_dict,savename)   


In [ ]:
bm25 = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/results_bm25_10.csv')

In [ ]:
bm25_docids = load('./results/ms_docids.json')

In [ ]:
calc_rtr_score(bm25, bm25_docids, './results/bm25_docids.json')

In [ ]:
monot5 = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/results_bm25_monot5_r10.csv') # 100 docs retrieved for each query.

In [ ]:
monot5_docids = load('./results/ms_docids.json')

In [ ]:
calc_rtr_score(monot5, monot5_docids, './results/monot5_docids.json')

In [ ]:
################ splade  ##################################

In [ ]:
splade = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/results_splade_10.csv') # 100 docs retrieved for each query.

In [ ]:
splade.head(2)

In [ ]:
splade_docids = load('./results/ms_docids.json')

In [ ]:
calc_rtr_score(splade, splade_docids, './results/splade_docids.json')

In [ ]:
colbert = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/result_colbert_10.csv').reset_index()

In [ ]:
colbert = colbert.rename(columns={'docid':'docno'})

In [ ]:
colbert.head(2)

In [ ]:
colbert_docids = load('./results/ms_docids.json')

In [ ]:
calc_rtr_score(colbert, colbert_docids, './results/colbert_docids.json')

In [ ]:
# import statistics

# def calc_stats(data):
#     values = list(data.values())

#     mean = statistics.mean(values)
#     std_dev = statistics.stdev(values) 
    
#     print(f"Mean: {mean}")
#     print(f"Standard Deviation: {std_dev}")

In [ ]:
# for v in bm25_docids.vavlues():
#     print(v)

In [ ]:
bm25_docids = load('./results/bm25_docids.json')

In [ ]:
monot5_docids = load('./results/monot5_docids.json')

In [ ]:
splade_docids = load('./results/splade_docids.json')

In [ ]:
colbert_docids = load('./results/colbert_docids.json')

In [ ]:
calc_stats(bm25_docids)

In [ ]:
calc_stats(monot5_docids)

In [ ]:
calc_stats(splade_docids)

In [ ]:
calc_stats(colbert_docids)

In [ ]:
def gini_coefficient(data_dict):
    # Extract values from the dictionary
    # values = list(data_dict.values())
    values = [int(i) for i in data_dict.values()]

    print(min(values))
    print(max(values))
    # Sort values in ascending order
    sorted_values = sorted(values)

    
    
    # Number of values
    n = len(sorted_values)
    
    # Check for zero total (to avoid division errors)
    if n == 0:
        return 0
    
    # Calculate the cumulative sums of the sorted values
    cumulative_sum = sum((i + 1) * val for i, val in enumerate(sorted_values))
    total = sum(sorted_values)
    
    # Calculate the Gini coefficient
    gini = 1 - (2 * cumulative_sum) / (n * total) + (1 / n)
    return gini


In [ ]:

def gini_coefficient2(data_dict):
    # values = list(data_dict.values())
    values = data_dict
    sort_list=sorted(values)
    sum_number=sum(sort_list)
    accm_list=sort_list.copy()
    for i in range(len(accm_list)-1):
        accm_list[i+1]=accm_list[i]+accm_list[i+1]
    return 1-(sum_number*0.5+sum(accm_list[:-1]))/(sum_number*len(sort_list)*0.5)



In [ ]:
# Calculate Gini coefficient

gini_value = gini_coefficient3(bm25_docids)
print(f"Gini Coefficient: {gini_value:.4f}")

In [ ]:
# Calculate Gini coefficient
gini_value = gini(monot5_docids)
print(f"Gini Coefficient: {gini_value:.4f}")
# Gini Coefficient: 0.9268

In [ ]:
# Calculate Gini coefficient
gini_value = gini(splade_docids)
print(f"Gini Coefficient: {gini_value:.4f}")
# Gini Coefficient: 0.9601

In [ ]:
# Calculate Gini coefficient
gini_value = gini(colbert_docids)
print(f"Gini Coefficient: {gini_value:.4f}")
# Gini Coefficient: 0.9260

In [ ]:
bm25_csv = pd.DataFrame.from_dict(bm25_docids, orient="index",columns=["score"])

In [ ]:
bm25_csv.to_csv('./results/bm25.csv')

In [ ]:
monot5_csv = pd.DataFrame.from_dict(monot5_docids, orient="index",columns=["score"])

In [ ]:
monot5_csv.to_csv('./results/monot5.csv')

In [ ]:
splade_csv = pd.DataFrame.from_dict(splade_docids, orient="index",columns=["score"])
splade_csv.to_csv('./results/splade.csv')


In [ ]:
colbert_csv = pd.DataFrame.from_dict(colbert_docids, orient="index",columns=["score"])
colbert_csv.to_csv('./results/colbert.csv')

In [ ]:
bm25_csv = pd.read_csv('./results/bm25.csv')




In [ ]:
bm25csv = pd.read_csv('./results/bm25.csv',columns=[''])

In [ ]:
bm25csv.head()

In [ ]:
bm25_csv = pd.read_csv('./results/bm25.csv')
column = bm25_csv.columns[1]
bm25_csv = bm25_csv[bm25_csv[column]>0]
bm25_csv.to_csv('./results/bm25_csv_pruned.csv', index=False)

monot5_csv = pd.read_csv('./results/monot5.csv')
column = monot5_csv.columns[1]
monot5_csv = monot5_csv[monot5_csv[column]>0]
monot5_csv.to_csv('./results/monot5_csv_pruned.csv', index=False)

splade_csv = pd.read_csv('./results/splade.csv')
column = splade_csv.columns[1]
splade_csv = splade_csv[splade_csv[column]>0]
splade_csv.to_csv('./results/splade_csv_pruned.csv', index=False)

colbert_csv = pd.read_csv('./results/colbert.csv')
column = colbert_csv.columns[1]
colbert_csv = colbert_csv[colbert_csv[column]>0]
colbert_csv.to_csv('./results/colbert_csv_pruned.csv', index=False)

In [ ]:
bm25_csv = pd.read_csv('./results/bm25.csv')
column = bm25_csv.columns[1]
bm25_csv = bm25_csv[bm25_csv[column] == 0]
bm25_csv.to_csv('./results/bm25_csv_with_0.csv', index=False)

monot5_csv = pd.read_csv('./results/monot5.csv')
column = monot5_csv.columns[1]
monot5_csv = monot5_csv[monot5_csv[column] == 0]
monot5_csv.to_csv('./results/monot5_csv_with_0.csv', index=False)

splade_csv = pd.read_csv('./results/splade.csv')
column = splade_csv.columns[1]
splade_csv = splade_csv[splade_csv[column] == 0]
splade_csv.to_csv('./results/splade_csv_with_0.csv', index=False)

colbert_csv = pd.read_csv('./results/colbert.csv')
column = colbert_csv.columns[1]
colbert_csv = colbert_csv[colbert_csv[column] == 0]
colbert_csv.to_csv('./results/colbert_csv_with_0.csv', index=False)

In [ ]:
bm25_csv.shape

In [ ]:
monot5_csv.shape

In [ ]:
splade_csv.shape

In [ ]:
colbert_csv.shape

In [ ]:
colbert_csv = pd.read_csv('./results/colbert_csv_pruned.csv')

In [ ]:
colbert_csv.sort_values(by='0',ascending=False).head(10)

In [ ]:
def G(v):
    v = np.array(v)
    bins = np.linspace(0., 100., 11)
    total = float(np.sum(v))
    yvals = [0]
    for b in bins[1:]:
        bin_vals = v[v <= np.percentile(v, b)]
        bin_fraction = (np.sum(bin_vals) / total) * 100.0
        yvals.append(bin_fraction)
    # perfect equality area
    pe_area = np.trapz(bins, x=bins)
    # lorenz area
    lorenz_area = np.trapz(yvals, x=bins)
    gini_val = (pe_area - lorenz_area) / float(pe_area)
    # return bins, yvals, gini_val
    return gini_val

In [ ]:
bm25_csv = pd.read_csv('./results/bm25_csv_pruned.csv',ignore_index=True)

In [ ]:
bm25_csv.head()

In [ ]:
bm25_csv = pd.read_csv('./results/bm25.csv')
column = bm25_csv.columns[0]
bm25_csv = bm25_csv[bm25_csv[column]>0]
# bm25_csv.to_csv('./results/bm25_csv_pruned.csv')
bm25_csv.head()

In [ ]:
scores = bm25_csv[bm25_csv.columns[1]].to_list()

In [ ]:
gini_value = gini_coefficient2(scores)
gini_value

In [ ]:
scores = monot5_csv[monot5_csv.columns[1]].to_list()
gini_value = gini_coefficient2(scores)
gini_value

In [ ]:
scores = splade_csv[splade_csv.columns[1]].to_list()
gini_value = gini_coefficient2(scores)
gini_value

In [ ]:
scores = colbert_csv[colbert_csv.columns[1]].to_list()
gini_value = gini_coefficient2(scores)
gini_value

In [ ]:
import statistics

def calc_stats(values):
    # values = list(data.values())

    mean = statistics.mean(values)
    std_dev = statistics.stdev(values) 
    
    print(f"Mean: {mean}")
    print(f"Standard Deviation: {std_dev}")

In [ ]:
calc_stats(bm25_csv[bm25_csv.columns[1]].to_list())

In [ ]:
calc_stats(monot5_csv[monot5_csv.columns[1]].to_list())

In [ ]:
calc_stats(splade_csv[splade_csv.columns[1]].to_list())

In [ ]:
calc_stats(colbert_csv[colbert_csv.columns[1]].to_list())

In [ ]:
bm25_csv.head(2)

In [ ]:
import ir_measures
from ir_measures import nDCG, MRR


def calc_metrics(df):
    # Prepare query-level results
    run = []
    for qid, group in df.groupby('qid'):
        for _, row in group.iterrows():
            input_data = [ir_measures.ScoredDoc(query_id=row['qid'], doc_id=row['docid'], score=10- row['rank'])]
            run.append(input_data)

    print(run)
    # qrel = [
    # ir_measures.Qrel(query_id=row['qid'], doc_id=row['docno'], relevance=row['label'])
    # for _, row in qrels.iterrows()
    # ]
    
    # # Calculate metrics
    # metrics = ir_measures.calc_aggregate([nDCG @ 10, MRR], qrel, run)
    
    # # Print metrics
    # print(f"nDCG@10: {metrics[nDCG @ 10]}")
    # print(f"MRR: {metrics[MRR]}")



In [ ]:
calc_metrics(bm25[:20])

In [ ]:
qls=qls[['query_id','doc_id','relevance']]

In [ ]:
qls[['query_id','doc_id']] = qls[['query_id','doc_id']].astype(str)

In [ ]:
qls['query_id'] = qls['query_id'].astype(str)

In [ ]:
qls.dtypes

In [ ]:
bm25_run[:10]

In [ ]:
bm25_run.info()

In [ ]:
# aggregated results
metrics = ir_measures.calc_aggregate([nDCG@10, MRR], qls, bm25_run[:10])
print(metrics)

In [ ]:
print()

In [ ]:
qrels.tail(10)

In [ ]:
bm25_run.head()

In [349]:
import ir_datasets
qrels = ir_datasets.load("msmarco-passage/dev").qrels_iter()

In [350]:
bm25 = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/results_bm25_10.csv', index_col=0)

In [351]:
bm25[:5]

,qid,docid,docno,rank,score,query,text
0,1048578,7187236,7187236,0,67.528593,cost of endless pools swim spa,The Endless Pool 15' endless pools swim spa Endless Pool Nightmare Buy A Big Spa Instead Great training pool
1,1048578,7471198,7471198,1,62.121928,cost of endless pools swim spa,"Cal Spas is the leading maker of home resort products. Cal Spas makes a great jetted swim spa with thoughtful attention paid to all design features. Read our Cal Spas Swim Spa Review. Dimesion One offers swim spas through their Aquatic Fitness Systems brand. Read our Dimension One Swim Spa Review. Endless Pools is one of the industry leaders in swim spas and counter-current pools. Known for their superior swim current, Endless Pools sell a range of swim spas and modular pools."
2,1048578,5365326,5365326,2,51.958492,cost of endless pools swim spa,"How much does a swim spa cost. 1 A top of the line, branded swim spa 9 feet by 16 feet with the bells and whistles could be $30,000 to $40,000. A number of value brands will sell swim spas in the $15,000 to $18,000 range. 2 CalSpas sells swim spas from $14,999 to $18,999 depending upon the product and configuration (May 2011). 3 Endless Pools sells swim spas from $7,500 to $28,900."
3,1048578,7187241,7187241,3,51.947947,cost of endless pools swim spa,"Disclaimer: Endless Pools (and all other swim spa manufacturers) are intentionally secretive about their pricing. All information in this article should be seen as a guide only, we make no claims to the accuracy of any endless pool prices on this page. All opinions are our own. We are not associated with any particular brand of swim spa in any way."
4,1048578,6802210,6802210,4,47.925985,cost of endless pools swim spa,"Endless Pools is one of the first companies to develop this Swim Spa, or “endless” swimming pool concept quite a number of years ago. Because of its modular design, it can be installed indoors in existing spaces. All of the components of an Original Endless Pool fit easily through doorways and down stairs."


In [352]:
bm25_run = bm25.rename(columns={'qid':'query_id','docid':'doc_id'})
del bm25_run['score']
bm25_run['score'] = 10-bm25_run['rank']
bm25_run = bm25_run[['query_id','doc_id','score']]
bm25_run[['query_id','doc_id']] = bm25_run[['query_id','doc_id']].astype(str)
ir_measures.calc_aggregate([nDCG@10, RR], qrels, bm25_run[:10])
# print(metrics)

{nDCG@10: 5.416351715858455e-06, RR: 1.9991923263001747e-06}

In [343]:
bm25_run.head()

,query_id,doc_id,docno,rank,query,text,score
0,1048578,7187236,7187236,0,cost of endless pools swim spa,The Endless Pool 15' endless pools swim spa Endless Pool Nightmare Buy A Big Spa Instead Great training pool,10
1,1048578,7471198,7471198,1,cost of endless pools swim spa,"Cal Spas is the leading maker of home resort products. Cal Spas makes a great jetted swim spa with thoughtful attention paid to all design features. Read our Cal Spas Swim Spa Review. Dimesion One offers swim spas through their Aquatic Fitness Systems brand. Read our Dimension One Swim Spa Review. Endless Pools is one of the industry leaders in swim spas and counter-current pools. Known for their superior swim current, Endless Pools sell a range of swim spas and modular pools.",9
2,1048578,5365326,5365326,2,cost of endless pools swim spa,"How much does a swim spa cost. 1 A top of the line, branded swim spa 9 feet by 16 feet with the bells and whistles could be $30,000 to $40,000. A number of value brands will sell swim spas in the $15,000 to $18,000 range. 2 CalSpas sells swim spas from $14,999 to $18,999 depending upon the product and configuration (May 2011). 3 Endless Pools sells swim spas from $7,500 to $28,900.",8
3,1048578,7187241,7187241,3,cost of endless pools swim spa,"Disclaimer: Endless Pools (and all other swim spa manufacturers) are intentionally secretive about their pricing. All information in this article should be seen as a guide only, we make no claims to the accuracy of any endless pool prices on this page. All opinions are our own. We are not associated with any particular brand of swim spa in any way.",7
4,1048578,6802210,6802210,4,cost of endless pools swim spa,"Endless Pools is one of the first companies to develop this Swim Spa, or “endless” swimming pool concept quite a number of years ago. Because of its modular design, it can be installed indoors in existing spaces. All of the components of an Original Endless Pool fit easily through doorways and down stairs.",6


In [ ]:
monot5 = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/results_bm25_monot5_r10.csv') 
monot5[:10]

In [ ]:
monot5_run = monot5.rename(columns={'qid':'query_id','docid':'doc_id'})
del monot5_run['score']
monot5_run['score'] = 10-monot5_run['rank']
monot5_run[['query_id','doc_id']] = monot5_run[['query_id','doc_id']].astype(str)
metrics = ir_measures.calc_aggregate([nDCG@10, RR], qrels, monot5_run)
print(metrics)

In [ ]:
splade = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/results_splade_10.csv') 
splade[:5]

In [ ]:
splade_run = splade.rename(columns={'qid':'query_id','docid':'doc_id'})
del splade_run['score']
splade_run['score'] = 10-splade_run['rank']
splade_run[['query_id','doc_id']] = splade_run[['query_id','doc_id']].astype(str)
metrics = ir_measures.calc_aggregate([nDCG@10, RR], qrels, splade_run)
print(metrics)

In [321]:
colbert = pd.read_csv('/nfs/datasets/cxj/retrievability-bias/result_colbert_10.csv').reset_index()

In [322]:
colbert.head()

,index,qid,docid,rank
0,0,13021,6728409,0
1,1,13021,7076490,1
2,2,13021,6911464,2
3,3,13021,7351852,3
4,4,13021,3257520,4


In [ ]:
colbert_run = colbert.rename(columns={'qid':'query_id','docid':'doc_id'})
# del colbert_run['score']
colbert_run['score'] = 10-colbert_run['rank']
colbert_run[['query_id','doc_id']] = colbert_run[['query_id','doc_id']].astype(str)
metrics = ir_measures.calc_aggregate([nDCG@10, RR], qrels, colbert_run)
print(metrics)